In [4]:
import joblib
import pandas as pd
import datetime
import numpy as np
import mysql.connector

In [17]:
df = pd.read_csv('July_15_90MinCutoff_Final_WithID.csv')
df = pd.read_csv('July_15_90MinCutoff_Final_WithID.csv')
youtube = pd.read_csv('YoutubeViews.csv')

mydb = mysql.connector.connect(
  host="cinesave-2.chxybcbl9we4.us-east-1.rds.amazonaws.com",
  user="admin",
  password="Cinesave1",
  database = "sample"
)

print(mydb)
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM movie_revenue_third")

myresult = mycursor.fetchall()
external = pd.DataFrame(myresult, columns = mycursor.column_names)

youtube = youtube.drop(columns = ['Unnamed: 0'])
external['name'] = external['name'].str.replace('\n', '')
external['in_collection'] = np.where(external['collection'] == '', 0, 1)
external = external.drop(columns = ['collection', 'budget', 'genre', 'popularity', 'release_date', 'revenue', 'production_company', 'earnings'])
external.columns = ['Title', 'runtime', 'vote_average', 'vote_count', 'in_collection']
df = pd.merge(df, external, on = 'Title', how = 'left')
df = df.drop(columns = ['vote_average'])
df['runtime'] = df['runtime'].fillna(value = 132)
df['in_collection'] = df['in_collection'].fillna(value = 0)
df['vote_count'] = df['vote_count'].fillna(value = 7391)
youtube['movie'] = youtube['movie'].str.replace('\n', '')
youtube['trailer views'] = youtube['trailer views'].str.replace(' views', '')
youtube['trailer views'] = youtube['trailer views'].str.replace(',', '')
youtube['trailer views'] = youtube['trailer views'].astype(int)
youtube.columns = ['Title', 'trailer_views']
df_new = pd.merge(df, youtube, how = 'left', on = 'Title')
df_na = df_new[df_new['trailer_views'].isna() == True]
df_new = df_new.dropna()
df = df_new
df = df.loc[df['Time Difference'] >= 0]
df = df.loc[df['Previous_Fullness'] != 0]
df = df.drop(columns = ['theater identifier', 'release date', 'Show_Datetime', 'Weekend', 'Night', 'Previous_Fullness', 'vote_count', 'in_collection', 'trailer_views', 'Week_Day', 'days_out', 'Time Taken', 'Time Difference'])


In [19]:
df.to_csv('movie_info.csv', index = False)

In [18]:
df

,id,Title,price,time,Show Date,Earnings,IMDB rating,Popularity,Release Date,Star Power,Actual_Fullness,runtime
10,41,1917,16.49,855,3/15/20,147587469,8.5,4,Release Date: 10 January 2020 (USA) See more »,10072,0.236842,119.0
11,62,1917,16.49,1035,3/15/20,147587469,8.5,4,Release Date: 10 January 2020 (USA) See more »,10072,0.463816,119.0
12,65,1917,16.49,855,3/15/20,147587469,8.5,4,Release Date: 10 January 2020 (USA) See more »,10072,0.325658,119.0
13,85,1917,16.49,1035,3/15/20,147587469,8.5,4,Release Date: 10 January 2020 (USA) See more »,10072,0.555921,119.0
14,103,1917,16.49,1215,3/15/20,147587469,8.5,4,Release Date: 10 January 2020 (USA) See more »,10072,0.585526,119.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1041,4203,The Lodge,16.49,975,3/16/20,1666564,6.6,279,Release Date: 16 January 2020 (Italy) See more »,10533,0.160256,108.0
1042,4234,The Lodge,16.49,975,3/16/20,1666564,6.6,279,Release Date: 16 January 2020 (Italy) See more »,10533,0.160256,108.0
1043,4241,The Lodge,16.49,975,3/16/20,1666564,6.6,279,Release Date: 16 January 2020 (Italy) See more »,10533,0.173077,108.0
1044,4247,The Lodge,16.49,975,3/16/20,1666564,6.6,279,Release Date: 16 January 2020 (Italy) See more »,10533,0.173077,108.0
